In [1]:
from casadi import *

**Problem**:

Wir wollen den eintretenden Strom in den Server begrenzen, **wenn nötig** und dem sendenden Server mitteilen, dass noch mehr geschickt werden kann, **wenn möglich**.
- $v_{in,req}$: gesendeter Strom, kann den Server überlasten und muss deshalb abgeschnitten werden, wenn:
- $v_{in,req}\geq v_{in,max}$, dabei ist $v_{in,max}$ eine Stellgröße.
- $v_{in}$: tatsächlich eintretender Strom

Wir wollen aber auch gleichzeitig erreichen, dass $v_{in,max}$ möglichst groß gewählt wird, wenn $v_{in,req}\leq v_{in,max}$. 


**Lösungsansatz:**

Hilfsvariablen einführen:

In [46]:
v_req = SX.sym('v_req',1,1)
delta_plus = SX.sym('delta_plus',1,1)
delta_minus = SX.sym('delta_minus',1,1)
v_max = SX.sym('v_max',1,1)

v_in = v_req-delta_minus
v_in_max = v_in+delta_plus

obj = -delta_plus+delta_minus

cons = vertcat(*[
    -delta_plus,
    -delta_minus,
    v_in-v_max,
    v_in_max-v_max
])

eq_cons = delta_plus*delta_minus

ub = np.zeros((5,1))
lb = -np.inf*np.ones((5,1))
lb[-1] =0

optim_dict = {
    'x': vertcat(delta_plus,delta_minus),
    'f': obj,
    'g': vertcat(cons, eq_cons),
    'p': vertcat(v_req, v_max)
}


In [47]:
S = nlpsol('S', 'ipopt', optim_dict)

In [50]:
S(ubg=ub,lbg=lb,p=np.array([3,3]), x0=np.array([0,0]))

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        4

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

{'f': DM(-2.48231e-09),
 'g': DM([-3.67924e-19, 0, 0, 0, 0]),
 'lam_g': DM([0.25059, 0.333334, 0.333334, 0.333334, -3.69536e+08]),
 'lam_p': DM([-0.666669, 0.666669]),
 'lam_x': DM([0, 0]),
 'x': DM([3.67924e-19, -2.48231e-09])}

In [69]:
a= [1, [2,3],4]